In [1]:
import openai
import time
import pandas as pd
from pathlib import Path
from tqdm import tqdm

APIキーは以下のURLから取得できます。  
https://platform.openai.com/account/api-keys  

【注意】  
<font color="Crimson">APIキーが流出すると他人に利用されて料金を請求されます！</font>  
絶対に共有してはいけません。  

In [2]:
# APIキーの読み込み
# キーを格納したテキストファイルを指定して読み込み
with open(f'{Path.home()}/.private_info/api_key.txt') as f:
    openai.api_key = f.read()

# ChatGPT APIのリクエスト送信制限について  
  
時間当たりのトークン量の制限とリクエスト回数の制限があり、どちらかでも引っかかるとエラーになる。  
[リミットについての詳細](https://platform.openai.com/docs/guides/rate-limits/overview)  

[トークンカウンター](https://platform.openai.com/tokenizer)  

In [3]:
# ChatGPT settings
def send_request(message: str, role: str, delay: float = None, **kwargs):
    """
    Delay a completion to avoid a rate limit of requests per min (RPM).
    Free trial users: 3 RPM
    Pay-as-you-go users: 3500 RPM
    """
    if delay:
        time.sleep(delay)
    messages = []
    # system role
    if role:
        messages.append(
            {'role': 'system', 'content': role},
        )
    # input message
    messages.append(
        {'role': 'user', 'content': message},
    )
    # send request
    chat_completion = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=messages,
        **kwargs
    )
    # get reply
    reply = chat_completion.choices[0].message.content
    return reply

# CSV処理関数の作成

In [4]:
def reply_to_csv(path: str,
                 message_col: str = 'message',
                 role_col: str = None,
                 delay: float = 3,
                 read_kwargs: dict = {},
                 gpt_kwargs: dict = {}) -> pd.core.frame.DataFrame:

    df = pd.read_csv(path, **read_kwargs)
    message_list = df.loc[:, message_col].values
    if role_col:
        role_list = df.loc[:, role_col].values
    else:
        role_list = [None] * len(message_list)
    assert len(message_list) == len(role_list), 'role列とmessage列の長さが異なります。揃えて再実行して下さい。'
    reply_list = []
    for role, message in zip(tqdm(role_list), message_list):
        reply = send_request(message=message, role=role, delay=delay, **gpt_kwargs)
        reply_list.append(reply)
    df['reply'] = reply_list
    df.to_csv(f'./data/{path[:-4]}_reply.csv', encoding='utf-8-sig', index=False)
    return df

In [5]:
df = reply_to_csv(path='./data/questions.csv',
                  message_col='message',
                  role_col='role',
                  delay=None,
                  read_kwargs=dict(encoding='utf-8-sig'),
                  gpt_kwargs=dict(temperature=0.7))

100%|██████████| 24/24 [00:26<00:00,  1.12s/it]
